In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE143272"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE143272"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE143272.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE143272.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE143272.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral blood expression profiles of patients with epilepsy receiving and not receiving antiepileptic drug monotherapy and their differential response to the treatment."
!Series_summary	"The aim here was to identify mRNA expression biomarkers associated with the disease epilepsy and the antiepileptic drug response. Gene expression profiles of drug-naïve patients with epilepsy were compared with that of healthy controls. The profiles were significantly different between the two groups as well as patients with different epilepsy types i.e. idiopathic, symptomatic and cryptogenic. Besides, patients showing differential response to antiepileptic monotherapies were also having differential blood gene expression profiles."
!Series_overall_design	"Total RNA obtained from peripheral blood samples of 34 drug-naïve patients with epilepsy and 57 followed-up patients showing differential response to antiepileptic drug monotherapy along with 50 healthy subj

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset appears to contain gene expression data
# The series title and summary mention "mRNA expression biomarkers" and "gene expression profiles"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Epilepsy):
# From the sample characteristics, we can infer epilepsy status from rows 2 and 3 (epilepsy type and treatment)
# We can consider someone as having epilepsy if they have an epilepsy type or are receiving treatment
trait_row = 2  # Using epilepsy type as our indicator

# Age:
# Age is available in row 0
age_row = 0

# Gender:
# Gender (Sex) is available in row 1
gender_row = 1

# 2.2 Data Type Conversion

# Trait (Epilepsy):
def convert_trait(value):
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # If epilepsy type is "-", this indicates a control subject without epilepsy
    if value == "-":
        return 0  # No epilepsy
    # If any type of epilepsy is mentioned, they have epilepsy
    elif value in ["Idiopathic", "Symptomatic", "Cryptogenic"]:
        return 1  # Has epilepsy
    else:
        return None  # Unknown

# Age:
def convert_age(value):
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)  # Convert to numeric
    except:
        return None  # Handle non-numeric or missing values

# Gender:
def convert_gender(value):
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None  # Unknown

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM4255766': [0.0, 26.0, 0.0], 'GSM4255767': [1.0, 28.0, 1.0], 'GSM4255768': [0.0, 29.0, 1.0], 'GSM4255769': [0.0, 28.0, 1.0], 'GSM4255770': [1.0, 32.0, 1.0], 'GSM4255771': [1.0, 27.0, 0.0], 'GSM4255772': [0.0, 22.0, 0.0], 'GSM4255773': [1.0, 16.0, 1.0], 'GSM4255774': [1.0, 14.0, 0.0], 'GSM4255775': [0.0, 25.0, 0.0], 'GSM4255776': [0.0, 20.0, 1.0], 'GSM4255777': [1.0, 18.0, 0.0], 'GSM4255778': [1.0, 24.0, 1.0], 'GSM4255779': [0.0, 40.0, 0.0], 'GSM4255780': [0.0, 40.0, 1.0], 'GSM4255781': [1.0, 20.0, 0.0], 'GSM4255782': [0.0, 29.0, 0.0], 'GSM4255783': [1.0, 21.0, 0.0], 'GSM4255784': [1.0, 28.0, 1.0], 'GSM4255785': [1.0, 22.0, 1.0], 'GSM4255786': [0.0, 24.0, 0.0], 'GSM4255787': [0.0, 38.0, 0.0], 'GSM4255788': [1.0, 23.0, 1.0], 'GSM4255789': [0.0, 48.0, 1.0], 'GSM4255790': [0.0, 34.0, 0.0], 'GSM4255791': [1.0, 20.0, 1.0], 'GSM4255792': [1.0, 10.0, 1.0], 'GSM4255793': [0.0, 35.0, 1.0], 'GSM4255794': [1.0, 15.0, 1.0], 'GSM4255795': [1.0, 17.0, 1.0], 'GSM4255796':

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE143272/GSE143272_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE143272/GSE143272_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (13165, 142)
First 20 gene/probe identifiers:
['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651296', 'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651336', 'ILMN_1651346', 'ILMN_1651347', 'ILMN_1651364', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403', 'ILMN_1651405', 'ILMN_1651429']


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers starting with "ILMN_", these are Illumina microarray probe IDs
# They are not human gene symbols and will need to be mapped to gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, 'ID' appears to be the probe ID and 'Symbol' contains gene names
# Display more samples from the Symbol column to better understand the format
print("\nSample of Symbol column (first 5 rows):")
if 'Symbol' in gene_annotation.columns:
    for i in range(min(5, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['Symbol'].iloc[i]}")

# Check the quality and completeness of the mapping
if 'Symbol' in gene_annotation.columns:
    non_null_symbols = gene_annotation['Symbol'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nSymbol column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation dataframe
# From the preview, we can see:
# - 'ID' column in the gene annotation contains the probe IDs (ILMN_*)
# - 'Symbol' column contains the gene symbols

# 2. Get a gene mapping dataframe by extracting the ID and Symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Check the mapping dataframe
print("\nGene mapping preview:")
print(preview_df(mapping_df, n=5))
print(f"Mapping dataframe shape: {mapping_df.shape}")

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the gene expression data
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
if gene_data.shape[0] > 0:
    print("First 5 gene symbols after mapping:")
    print(gene_data.index[:5].tolist())

    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("Warning: No gene symbols were mapped successfully.")



Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}
Mapping dataframe shape: (44837, 2)

Gene expression data after mapping:
Shape: (9221, 142)
First 5 gene symbols after mapping:
['A2LD1', 'AADACL1', 'AAGAB', 'AAK1', 'AAMP']


Gene expression data saved to ../../output/preprocess/Epilepsy/gene_data/GSE143272.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (9221, 142)
Gene data shape after normalization: (8978, 142)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE143272.csv
Extracted clinical data shape: (3, 142)
Preview of clinical data (first 5 samples):
          GSM4255766  GSM4255767  GSM4255768  GSM4255769  GSM4255770
Epilepsy         0.0         1.0         0.0         0.0         1.0
Age             26.0        28.0        29.0        28.0        32.0
Gender           0.0         1.0         1.0         1.0         1.0
Clinical data saved to ../../output/preprocess/Epilepsy/clinical_data/GSE143272.csv
Gene data columns (first 5): ['GSM4255766', 'GSM4255767', 'GSM4255768', 'GSM4255769', 'GSM4255770']
Clinical data columns (first 5): ['GSM4255766', 'GSM4255767', 'GSM4255768', 'GSM4255769', 'GSM4255770']
Found 142 common samples between gene and clinical data
Initial linked data shape: (142, 8981)
Preview of linked data (first 5 rows, first 5 columns):
            Epilepsy   Age  Gender     AAGAB      AAK1
GSM4255766       0.0  26.0     0.0  6.166417  5.549871
GSM4

Linked data shape after handling missing values: (142, 8981)
For the feature 'Epilepsy', the least common label is '0.0' with 51 occurrences. This represents 35.92% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 19.25
  50% (Median): 23.0
  75%: 28.0
Min: 10.0
Max: 48.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 62 occurrences. This represents 43.66% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Epilepsy/GSE143272.csv
